In [1]:
import numpy as np

In [15]:
""" Problem 1,2,3,4,5,6"""

class LinOpt:
    
    def __init__(self, c, A, b):
        
        x = np.zeros_like(c)
        if np.any(A.dot(x) > b):
            raise ValueError("Problem is not feasible at the origin")
        else:
            self.c = c
            self.A = A
            self.b = b
        
        #track variables
        m, n = A.shape
        self.L = list(range(n, n + m)) + list(range(n))
        
    def tableau(self):
        m,n = self.A.shape
        Im = np.eye(m)
        Abar = np.concatenate([self.A, np.eye(m)], axis = 1)
        cbar =  - np.concatenate([self.c, np.zeros(m)])
        col1 =  np.concatenate((np.array([0]), self.b))
        col2 =  np.vstack((cbar, Abar))
        col3 = np.concatenate((np.array([1]), np.zeros_like(self.b)))
        self.T = np.column_stack((col1, col2, col3))
        return self.T
        
    def pivot(self):
        m,n = self.T.shape
        index_col = 0
        while self.T[0,index_col] >= 0:
            index_col+=1
        if not np.any(self.T[:,index_col] > 0):
            raise ValueError("Cannot use Bland's Method")
        else:
            ratio = np.zeros(m)
            ratio = self.T[:, 0]/self.T[:,index_col]
            ratio[ratio <= 0] = np.inf
            index_row = np.where(ratio == min(ratio))[0][0]
        self.row = index_row
        self.col = index_col
        return self.row, self.col
    
    def pivot_op(self):
        row, col = self.row, self.col
        l1 = row-1
        l2 = self.L.index(col-1)
        self.L[l1], self.L[l2] = self.L[l2], self.L[l1]
        self.T[row,:] = self.T[row,:]/self.T[row, col]
        m, n = self.T.shape
        for i in range(m):
            if i == row:
                self.T[i,:] = self.T[i,:]
            else:
                coef = self.T[i, col]/self.T[row, col]
                self.T[i,:] = self.T[i,:]- coef* self.T[row,:]
        return self.L, self.T
    
    def solve(self):
        self.tableau()
        while np.any(self.T[0, 1:] < 0):
            self.pivot()
            self.pivot_op()
        optimal = self.T[0,0]
        return optimal

In [16]:
A = np.array([[1,-1],[3,1],[4,3]])
b = np.array([2,5,7])
c = np.array([3,2])
solver = LinOpt(c, A, b)
solver.tableau()
solver.pivot()
sol = solver.solve()
print(sol)

(5.2000000000000002, {2: 0.59999999999999964, 0: 1.6000000000000001, 1: 0.19999999999999982}, {4: 0})


In [17]:
""" Problem 7 """
data = np.load("/Users/Deehrlich/Desktop/BootCamp2017/Computation/Wk4_DifIntOpt/productMix.npz")
A = data['A']
p = data['p']
m = data['m']
d = data['d']
A = np.vstack([A, np.eye(A.shape[1])])
b = np.concatenate((m,d))
solver = LinOpt(p, A, b)
solver.tableau()
solver.pivot()
solver.pivot_op()
sol = solver.solve()
print(sol)

(7453.5964912280706, {1: 6.1929824561403484, 3: 1.7894736842105292, 6: 0.96596491228070158, 0: 10.0, 8: 13.807017543859653, 2: 12.0, 10: 8.2105263157894708}, {4: 0, 9: 0, 5: 0})


/Users/Deehrlich/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:39: RuntimeWarning: divide by zero encountered in true_divide
